In [ ]:
import warnings

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from Functions import pmfg
import seaborn as sns

import Classes

warnings.filterwarnings('ignore')
%matplotlib inline


df = pd.read_excel("Credit risk exposures and exposures to sovereigns a.xlsx", 
                   header=1, index_col=0)
df = df.iloc[:, 0:8]
df = df.drop("4. Total retail", axis = 1)
df.columns = ["1_Sov", "2_Fin", "3_Corp", "4-1_Ret_Res", "4-2_Ret_Rev", "4-3_Ret_SME", "5_CRE"]


bank_names_df = pd.read_csv("BankCodes.csv", index_col=0).iloc[:, 0].to_frame("Name")
bank_cap_df = pd.read_excel("Tier-1 CRs 2010.xlsx", 
                       header=2, index_col=1).iloc[:, 1].to_frame("Capital")
df = pd.concat([df, bank_names_df, bank_cap_df], axis=1)

sd_df = pd.read_excel("EU_SOV_Debt_Bank_Net_2011.xlsx", header=0, index_col=0).T
sd_df.loc[:, u"XX"] = np.maximum(df.iloc[:, 0] - sd_df.apply(np.sum, axis=1), 0)
sd_df.columns = sd_df.columns.rename("SD_Name")
sd_names = [sd.encode('ascii', 'ignore') for sd in sd_df.columns]


spread = [20, 55, 60, 130, 30, 5, 15, 40, 115, 10, 40, 25, 1400, 80, 315, 35,
          100, 20, 55, 40, 55, 50, 100, 20, 10, 50, 400, 70, 10, 30, 25, 20, 50]
spread_df = pd.DataFrame(data=spread, index=sd_names, columns=["spread"])
spread_df.loc[:, "Q"] = spread_df.loc[:, "spread"].apply(Classes.Q)

geo_df = sd_df.div(sd_df.apply(np.sum, axis=1), axis=0)

In [ ]:
world = Classes.World(df, geo_df, spread_df, 0.4, Classes.RiskWeights("min"))
assets = world.getAssets()
banks = world.getBanks()

evo_a, evo_b = world.runShock(banks, assets, "fin", ["DE", "FR"], 1.25)
evo_b.filter(like="R_").T.plot(figsize=(6,10), legend=False)

In [ ]:
world = Classes.World(df, geo_df, spread_df, 0.8, Classes.RiskWeights("min"))
assets = world.getAssets()
banks = world.getBanks()

evo_a, evo_b = world.runShock(banks, assets, "fin", ["DE", "FR"], 1.25)
evo_b.filter(like="R_").T.plot(figsize=(6,10), legend=False)

In [ ]:
evo_a.filter(like="r_").T.plot(figsize=(6,10), legend=False)

In [ ]:
world = Classes.World(df, geo_df, spread_df, 0.4, Classes.RiskWeights("min"))
assets = world.getAssets()
banks = world.getBanks()

evo_a, evo_b = world.runShock(banks, assets, "sov", ["DE", "FR"], 50)
evo_b.filter(like="R_").T.plot(figsize=(6,10), legend=False)

In [ ]:
world = Classes.World(df, geo_df, spread_df, 0.8, Classes.RiskWeights("min"))
assets = world.getAssets()
banks = world.getBanks()

evo_a, evo_b = world.runShock(banks, assets, "sov", ["DE", "FR"], 50)
evo_b.filter(like="R_").T.plot(figsize=(6,10), legend=False)

In [ ]:
world = Classes.World(df, geo_df, spread_df, 0.4, Classes.RiskWeights("min"))
assets = world.getAssets()
banks = world.getBanks()

evo_a, evo_b = world.runShock(banks, assets, "sov", ["GR", "PT"], 50)
evo_b.filter(like="R_").T.plot(figsize=(6,10), legend=False)

In [ ]:
world = Classes.World(df, geo_df, spread_df, 0.8, Classes.RiskWeights("min"))
assets = world.getAssets()
banks = world.getBanks()

evo_a, evo_b = world.runShock(banks, assets, "sov", ["GR", "PT"], 50)
evo_b.filter(like="R_").T.plot(figsize=(6,10), legend=False)

Increase of the risk factors when we shock sov debt in GR, PT, and it spreads highly to other sectors

In [ ]:
evo_a.filter(like="r_").T.plot(figsize=(6,10), legend=False)

In [ ]:
world = Classes.World(df, geo_df, spread_df, 0.8, Classes.RiskWeights("min"))
assets = world.getAssets()
banks = world.getBanks()

evo_a, evo_b = world.runShock(banks, assets, "fin", ["DE", "FR"], 1.25)

In [ ]:
res = []
for sp in np.arange(0.1, 1.1, 0.1):
    for sh in np.arange(1.25, 4.25, 0.25):
        world = Classes.World(df, geo_df, spread_df, sp, Classes.RiskWeights("min"))
        assets = world.getAssets()
        banks = world.getBanks()
        evo_a, evo_b = world.runShock(banks, assets, "fin", ["DE", "FR"], sh)
        res.append(
            [sp, sh, 1 - np.mean(evo_b.filter(like="R").iloc[:,-1])/np.mean(evo_b.filter(like="R").iloc[:,0])])

In [ ]:
x_df = pd.DataFrame(data=res, columns=["sp", "sh", "x"])
x_df.plot(kind='scatter', x='sp', y='x')